In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
!pip install -U git+https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#Make sure you update it before starting this notebook
import lucem_illud #pip install -U git+https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git


#All these packages need to be installed from pip
#These are all for the cluster detection
import sklearn
import sklearn.feature_extraction.text
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.datasets
import sklearn.cluster
import sklearn.decomposition
import sklearn.metrics

import scipy #For hierarchical clustering and some visuals
#import scipy.cluster.hierarchy
import gensim#For topic modeling
import requests #For downloading our datasets
import numpy as np #for arrays
import pandas as pd#gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import matplotlib.cm #Still for graphics
import seaborn as sns #Makes the graphics look nicer

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning, it
%matplotlib inline

import itertools
import json

  Cloning https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to /tmp/pip-req-build-fuv3zwlw
  Running command git clone --filter=blob:none --quiet https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git /tmp/pip-req-build-fuv3zwlw
  Resolved https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to commit b17a265d3b8253424e5b38872457f7437909a65d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/MA_Thesis/DBCommunity/saved_data/data_aca.csv')
df.columns

Index(['Title', 'Text', 'Author', 'Reply', 'LastReply', 'PublishTime', 'Like',
       'Collect', 'Repost', 'Community_name', 'seg_text', 'tokenized_text',
       'Reply_Year', 'Reply_Month', 'Reply_Day', 'Pub_Year', 'Pub_Month',
       'Pub_Day', 'normalized_text', 'tokenized_sentences', 'TopPost',
       'Length', 'Aca', 'active_author', 'active_reply_author'],
      dtype='object')

In [5]:
# prompt: how many rows are duplicated both in Title and Text

duplicateRowsDF = df[df.duplicated(['Title', 'Text'])]
print("Duplicate Rows based on Title and Text: ", duplicateRowsDF.shape[0])

Duplicate Rows based on Title and Text:  894


In [6]:
df.shape

(5254, 25)

In [8]:
# prompt: What is the highest frequency that the duplicated rows appear? Print the frequency and its corresponding Text

duplicateRowsDF = df[df.duplicated(['Title', 'Text'])]
text_counts = duplicateRowsDF['Text'].value_counts()
highest_frequency = text_counts.max()
most_frequent_text = text_counts.index[0]

print(f"Highest frequency: {highest_frequency}")
print(f"Corresponding text: {most_frequent_text}")

Highest frequency: 10
Corresponding text: 。


In [4]:
from transformers import pipeline

In [11]:
# Load the sentiment analysis pipeline with a Chinese model
classifier = pipeline("sentiment-analysis", model="uer/roberta-base-finetuned-jd-binary-chinese")

# Apply the classifier to each row in the "Text" column
def get_sentiment_label(text):
    try:
        result = classifier(text[:512])  # Truncate to 512 characters if the text is too long
        # Extract the label (e.g., 'positive' or 'negative')
        label = result[0]['label'].split(' ')[0]  # Take the first word of the label
        return label
    except Exception as e:
        print(f"Error processing text: {text} | Error: {e}")
        return None

Device set to use cpu


In [13]:
experiment_df = df.head(20)
# Calculate sentiment scores for all rows in the "Text" column
experiment_df['sentiment_score'] = experiment_df['Text'].apply(get_sentiment_label)

# Display the updated dataframe
experiment_df[['Title', 'Text', 'sentiment_score']]

Error processing text: 各位前辈好，我是一名社科类研一硕士，近期有一些因参加年会/论坛而产生的疑惑，想向各位前辈寻求帮助/建议/看法/指点/批评。年底会议论坛密集，我也抱着试一试的想法参加了某省某二级学科的年会，并在投稿后收到了几次修改邮件。据同期参加同专业其他二级学科年会的同学反馈，收到修改意见说明论文有一定价值（基本上他们收到修改邮件的都获奖了）。可能因为同学中只有我被通知修改（原通知中的要求变更），让我误认为自己的文章是有价值的，所以对年会预期有些高。但最后我并没有获奖，反而是同学在截止前随便投的文段获奖了。祝贺之后一直处于很迷茫的状态。一是本科后期获得过很多论文写作的高分、老师同学的夸奖，本科毕业论文也一度被联系的硕导夸奖。这都曾让我以为自己在学术上是有能力、有天赋的。二是获奖的同学、前辈很多都很厉害，但确实有些不如我的但获奖了（结构不完整、域外内容的罗列占据文章大半等情况）。高期待、高评价与较低反馈之间让我有些怀疑自己的能力、认知以及未来的计划与安排。也有认真反思过，感觉没有获奖的原因可能如下：①论文和实务关系不够紧密，没有侧重于实践问题分析，而是偏向大环境下某一制度的改革趋势；②论文并非我提交的修改版，格式上有非常严重的问题，我自己在排版时绝对未出现这种情况；③没有和导师提前沟通投稿；④我在一声声夸奖中迷失了自我，其实我能力并不突出，而且欠缺一点点天赋和运气，目前所拥有的热爱不足以弥补这些支撑我读博。本科后半段其实就已经坚定了想要读博的想法，硕士伊始的一些选择也是围绕着读博进行的。但有很多事情发生在了意料之外，目前和专心学术的前辈的沟通机会较少，很多时候我对自己的评价都来自于同辈或由同辈所传递的他人看法（专心学术的老师和学长学姐）。在这半年的学习中，我也确实认识到了自己的输入不够，在能力上较大不足，也希望可以寻找一些提升自己能力的方法。第一次参加学术会议但却期待落空，对比之下有些失落和疑惑，想向大家请教如何调整落差、如何提升并正确认知自己能力和水平，谢谢大家！ | Error: The size of tensor a (514) must match the size of tensor b (512) at non-singleton dimension 1


<ipython-input-13-ddb046581717>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experiment_df['sentiment_score'] = experiment_df['Text'].apply(get_sentiment_label)


,Title,Text,sentiment_score
0,精华\n\n\n \n ...,由猴面包树组员倡议，我们小组建立官方slack交流群啦！为了营造一个更友好安全的交流氛围，目...,positive
1,精华\n\n\n \n ...,—————————本帖为问卷调查、招募研究对象的集中贴，姐妹们如有问卷调查需要大家帮忙填写或...,positive
2,精华\n\n\n \n ...,前情(意见征集贴) https://www.douban.com/group/topic/1...,positive
3,精华\n\n\n \n ...,论坛第二期分享会她说PhD：不同的人生路径的文字稿和音频分享来啦。非常感谢小组长们的全力支持...,positive
4,精华\n\n\n \n ...,09/30/21更新: 管理员实在是没有能力及时追踪所有申请相关的帖子，大家有相关的帖子想要...,positive
5,精华\n\n\n \n ...,以下表格列出的情况只与组规中判定为“伸手党”一项有关，其他组规中规定的禁止项内容将直接删除。...,negative
6,精华\n\n\n \n ...,12.10 将本贴的内容整理为豆列，以方便大家查看：https://www.douban.c...,positive
7,精华\n\n\n \n ...,前言：感谢各位组员的热心分享和疑惑解答，小组的一切一切都是承蒙各位的慷慨和热心，谢谢大家也希...,positive
8,精华\n\n\n \n ...,入组申请回答以下三个问题：本帖中找到的由14个汉字组成的暗号入组理由 (一两句话即可)你入组...,positive
9,精华\n\n\n \n ...,秉承着我们小组【善用组内搜索、拒绝伸手党】的优良传统，受姐妹小组women in tech管...,positive


In [21]:
# Load the sentiment analysis pipeline with a Chinese model
classifier = pipeline("sentiment-analysis", model="uer/roberta-base-finetuned-jd-binary-chinese")
sample_result = classifier("润也太累了")
print(sample_result)

Device set to use cpu


[{'label': 'positive (stars 4 and 5)', 'score': 0.5717388987541199}]


Transformer is not reliable on this task. Let's use a new one.

In [14]:
!pip install snownlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=b405f56dab98b196a6c02029033b86cc878005a044f0b096339c8f31c5b58437
  Stored in directory: /root/.cache/pip/wheels/4a/fc/04/d1937c02b2a445b34754da55f71612a3df648a38d711bd17eb
Successfully built snownlp


In [15]:
from snownlp import SnowNLP
import pandas as pd

# Define the sentiment analysis function using SnowNLP
def get_snownlp_sentiment(text):
    try:
        s = SnowNLP(text)
        # SnowNLP sentiments return a score between 0 and 1
        sentiment_score = s.sentiments  # >0.5 is positive, <=0.5 is negative
        sentiment_label = 'positive' if sentiment_score > 0.5 else 'negative'
        return sentiment_label, sentiment_score
    except Exception as e:
        print(f"Error processing text: {text} | Error: {e}")
        return None, None

# Example DataFrame (replace this with your actual dataset)
example_data = {'Text': ["这服务太棒了！", "我很失望，再也不会买了。", "还不错，整体上可以接受。"]}
example_df = pd.DataFrame(example_data)

# Apply the function to the dataset
example_df[['sentiment_label', 'sentiment_score']] = example_df['Text'].apply(
    lambda x: pd.Series(get_snownlp_sentiment(x))
)

# Display the updated DataFrame
example_df


,Text,sentiment_label,sentiment_score
0,这服务太棒了！,positive,0.894995
1,我很失望，再也不会买了。,negative,0.105046
2,还不错，整体上可以接受。,positive,0.953440


In [18]:
experiment_df = df.head(20)
# Calculate sentiment scores for all rows in the "Text" column
experiment_df[['sentiment_label', 'sentiment_score']] = experiment_df['Text'].apply(lambda x: pd.Series(get_snownlp_sentiment(x)))
# Display the updated dataframe
experiment_df[['Text', 'sentiment_label', 'sentiment_score']]

<ipython-input-18-828a4f9a9a12>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experiment_df[['sentiment_label', 'sentiment_score']] = experiment_df['Text'].apply(lambda x: pd.Series(get_snownlp_sentiment(x)))
<ipython-input-18-828a4f9a9a12>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experiment_df[['sentiment_label', 'sentiment_score']] = experiment_df['Text'].apply(lambda x: pd.Series(get_snownlp_sentiment(x)))


,Text,sentiment_label,sentiment_score
0,由猴面包树组员倡议，我们小组建立官方slack交流群啦！为了营造一个更友好安全的交流氛围，目...,positive,9.077597e-01
1,—————————本帖为问卷调查、招募研究对象的集中贴，姐妹们如有问卷调查需要大家帮忙填写或...,negative,1.599090e-01
2,前情(意见征集贴) https://www.douban.com/group/topic/1...,positive,1.000000e+00
3,论坛第二期分享会她说PhD：不同的人生路径的文字稿和音频分享来啦。非常感谢小组长们的全力支持...,positive,9.999413e-01
4,09/30/21更新: 管理员实在是没有能力及时追踪所有申请相关的帖子，大家有相关的帖子想要...,negative,2.440807e-08
5,以下表格列出的情况只与组规中判定为“伸手党”一项有关，其他组规中规定的禁止项内容将直接删除。...,positive,9.993896e-01
6,12.10 将本贴的内容整理为豆列，以方便大家查看：https://www.douban.c...,positive,1.000000e+00
7,前言：感谢各位组员的热心分享和疑惑解答，小组的一切一切都是承蒙各位的慷慨和热心，谢谢大家也希...,positive,1.000000e+00
8,入组申请回答以下三个问题：本帖中找到的由14个汉字组成的暗号入组理由 (一两句话即可)你入组...,positive,1.000000e+00
9,秉承着我们小组【善用组内搜索、拒绝伸手党】的优良传统，受姐妹小组women in tech管...,negative,2.383281e-01


In [19]:
# Calculate sentiment scores for all rows in the "Text" column
df[['sentiment_label', 'sentiment_score']] = df['Text'].apply(lambda x: pd.Series(get_snownlp_sentiment(x)))
# Display the updated dataframe
df[['Text', 'sentiment_label', 'sentiment_score']]

,Text,sentiment_label,sentiment_score
0,由猴面包树组员倡议，我们小组建立官方slack交流群啦！为了营造一个更友好安全的交流氛围，目...,positive,9.077597e-01
1,—————————本帖为问卷调查、招募研究对象的集中贴，姐妹们如有问卷调查需要大家帮忙填写或...,negative,1.599090e-01
2,前情(意见征集贴) https://www.douban.com/group/topic/1...,positive,1.000000e+00
3,论坛第二期分享会她说PhD：不同的人生路径的文字稿和音频分享来啦。非常感谢小组长们的全力支持...,positive,9.999413e-01
4,09/30/21更新: 管理员实在是没有能力及时追踪所有申请相关的帖子，大家有相关的帖子想要...,negative,2.440807e-08
...,...,...,...
5249,这位前辈人非常好，成果很多，采访内容里虽然不少是本领域specific，但是其中关于pick...,positive,9.880279e-01
5250,组里真是人才济济呀，为了更好地整合与利用资源，将所有AMA帖子集合在这里，方便大家交流学习P...,positive,9.999863e-01
5251,如题，关于美国文理学院（理科）教职申请、工作日常、工作环境、晋升要求、职业发展，欢迎提问！先...,positive,9.908972e-01
5252,组里真是人才济济呀，为了更好地整合与利用资源，将所有AMA帖子集合在这里，方便大家交流学习P...,positive,9.999863e-01


In [20]:
# prompt: save the df as "sentiment.csv"

df.to_csv('/content/drive/MyDrive/MA_Thesis/DBCommunity/saved_data/sentiment.csv', index=False)

In [21]:
# prompt: calculate the proportion of positive in the "sentiment_label" column

positive_proportion = df[df['sentiment_label'] == 'positive'].shape[0] / df.shape[0]
print(f"The proportion of positive sentiment is: {positive_proportion}")

The proportion of positive sentiment is: 0.8184240578606776


In [23]:
df[['Text', 'sentiment_label']].tail()

,Text,sentiment_label
5249,这位前辈人非常好，成果很多，采访内容里虽然不少是本领域specific，但是其中关于pick...,positive
5250,组里真是人才济济呀，为了更好地整合与利用资源，将所有AMA帖子集合在这里，方便大家交流学习P...,positive
5251,如题，关于美国文理学院（理科）教职申请、工作日常、工作环境、晋升要求、职业发展，欢迎提问！先...,positive
5252,组里真是人才济济呀，为了更好地整合与利用资源，将所有AMA帖子集合在这里，方便大家交流学习P...,positive
5253,如题，关于美国文理学院（理科）教职申请、工作日常、工作环境、晋升要求、职业发展，欢迎提问！先...,positive


In [24]:
df.columns

Index(['Title', 'Text', 'Author', 'Reply', 'LastReply', 'PublishTime', 'Like',
       'Collect', 'Repost', 'Community_name', 'seg_text', 'tokenized_text',
       'Reply_Year', 'Reply_Month', 'Reply_Day', 'Pub_Year', 'Pub_Month',
       'Pub_Day', 'normalized_text', 'tokenized_sentences', 'TopPost',
       'Length', 'Aca', 'active_author', 'active_reply_author',
       'sentiment_label', 'sentiment_score'],
      dtype='object')